<a href="https://www.kaggle.com/code/harshaltambat/ai-personalized-diet-planner-agent?scriptVersionId=282687128" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [8]:
import requests
import json
import time
import random
import os
from kaggle_secrets import UserSecretsClient


# ==============================================================================
# AI PERSONALIZED DIET PLANNER AGENT
# ==============================================================================
# This notebook script creates a specialized, multi-stage AI agent designed to 
# gather detailed user metrics and generate a personalized 7-day diet plan.
#
# KEY ARCHITECTURAL CONCEPTS DEMONSTRATED:
#
# 1. Sequential Agents / State Management:
#    - The consultation is broken into 6 explicit, mandatory steps defined in 
#      QUESTION_SEQUENCE.
#    - The agent uses a 'current_step' variable to track progress and ensures 
#      questions are asked strictly in order, creating a highly structured user 
#      experience (Concept 1).
#
# 2. Tools (Built-in: Google Search Grounding):
#    - The final plan generation step explicitly uses Google Search to ground 
#      calorie and macro recommendations in real-time nutritional science and 
#      latest dietary standards (Concept 2).
#
# 3. Sessions & Memory (Context Compaction):
#    - Instead of feeding the entire, verbose chat history to the LLM for the 
#      final report, the user's key answers are collected and summarized via 
#      'compact_context()'.
#    - This compacted profile ensures the final plan is generated efficiently, 
#      saving tokens, and minimizing conversational noise (Concept 3).
#
# EXECUTION FLOW:
# 1. Initialization: The script sets up API keys and question lists.
# 2. Conversation Loop: The agent iterates through the 6 steps, storing key data 
#    in memory ('user_answers').
# 3. Finalization: Once all steps are complete, the 'user_answers' are compacted 
#    and the LLM is prompted (with Google Search enabled) to generate the final 
#    detailed plan.
# ==============================================================================

# --- Configuration ---
# NOTE: Replace 'YOUR_API_KEY' with your actual Gemini API key.
GOOGLE_API_KEY = "GOOGLE_API_KEY" 
try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

API_URL = f"https://generativelanguage.googleapis.com/v1beta/models/{MODEL_NAME}:generateContent?key={GOOGLE_API_KEY}"

# --- Agent State & Configuration Data ---

# Defines the specific sequence of questions the agent must ask.
# The agent will move through these steps sequentially.
QUESTION_SEQUENCE = [
    "Initial Metrics (Age, Height, Weight, Gender)",
    "Main Goal (Loss, Gain, Maintenance, Health)",
    "Activity Level (Sedentary, Lightly, Moderately, Very Active)",
    "Dietary Restrictions (Allergies, Vegan, Keto, etc.)",
    "Preferences & Dislikes (2-3 of each)",
    "Cooking Style (Time, Prep vs. Fresh)"
]

# This array stores the clean user input for context compaction (user answers only).
user_answers = {}
current_step = 0 # Tracks which question is next (0 to 5)

# --- System Prompts ---

# 1. System instruction for the conversational mode (Q&A phase)
CONVERSATION_SYSTEM_INSTRUCTION = """
    You are a friendly, certified AI Nutritionist and Diet Planner. 
    Your current task is to guide the user through a 6-step data gathering process.

    **Rules:**
    1. Acknowledge the user's previous answer in a concise and encouraging way.
    2. Immediately ask the NEXT question in the sequence.
    3. You MUST stop asking questions once all 6 pieces of information have been gathered.
"""

# 2. System instruction for the final plan generation mode (uses grounding)
PLAN_GENERATION_PROMPT = """
    You are a professional, data-driven AI Nutritionist. 
    Your task is to analyze the user's compacted profile data and generate a comprehensive, personalized 7-day diet and wellness plan. 
    
    **CRITICAL:** You must use Google Search grounding to ensure all nutritional recommendations (especially calorie and macro targets) are aligned with current standards for the user's profile.

    **User Profile (Compacted Context):**
    {compacted_context}

    **Final Plan Structure (Use Markdown, be detailed and encouraging):**
    * **Section 1: Summary & Estimated Metrics (Estimated Calorie Intake, Macro Goals: Protein/Fats/Carbs).** (Grounding is required for these estimates)
    * **Section 2: Personalized 7-Day Meal Plan (Provide a sample 3-day menu for breakfast, lunch, and dinner, factoring in preferences).**
    * **Section 3: Hydration and Activity Recommendations.**
    * **Section 4: Key Wellness Tips (3 actionable steps for their specific goal).**
"""

# --- Memory Management (Context Compaction) ---

def compact_context():
    """
    Concept 3: Context Compaction (Sessions & Memory)
    Compacts the verbose Q&A chat history into a single, structured string.
    """
    if not user_answers:
        return "No user data collected."

    compaction = "User Profile Summary:\n"
    for key, value in user_answers.items():
        compaction += f"- {key}: {value}\n"
    
    return compaction

# --- Core API Logic (Sequential Agents & Tools) ---

def generate_content_with_retry(history, system_prompt, use_grounding=True, max_retries=5):
    """
    Makes the Gemini API call with exponential backoff and retries.
    Concept 2: Tools (Web Grounding) - Conditional inclusion of Google Search.
    """
    payload = {
        "contents": history,
        "systemInstruction": {
            "parts": [{"text": system_prompt}]
        },
        "generationConfig": {
            "temperature": 0.6 
        }
    }

    if use_grounding:
        # Enable Google Search for grounded responses on the final plan
        payload["tools"] = [{"google_search": {}}]

    headers = {'Content-Type': 'application/json'}
    
    for attempt in range(max_retries):
        try:
            response = requests.post(API_URL, headers=headers, data=json.dumps(payload))
            response.raise_for_status()

            result = response.json()
            
            # Extract the text
            text = result.get('candidates', [{}])[0].get('content', {}).get('parts', [{}])[0].get('text')
            
            # Extract grounding sources if used
            sources = []
            if use_grounding:
                grounding_metadata = result.get('candidates', [{}])[0].get('groundingMetadata')
                if grounding_metadata and grounding_metadata.get('groundingAttributions'):
                    sources = [
                        f"[{i+1}] {a.get('web', {}).get('title')} ({a.get('web', {}).get('uri')})"
                        for i, a in enumerate(grounding_metadata['groundingAttributions'])
                        if a.get('web')
                    ]
            
            if text:
                return text, sources
            else:
                raise ValueError("API response was missing content.")

        except requests.exceptions.RequestException as e:
            if attempt < max_retries - 1:
                backoff_time = (2 ** attempt) * 1 + random.uniform(0, 1)
                print(f"Transient error occurred: {e}. Retrying in {backoff_time:.2f} seconds...")
                time.sleep(backoff_time)
            else:
                print(f"API Call failed after {max_retries} attempts.")
                raise e
        except Exception as e:
            print(f"An unexpected error occurred: {e}")
            raise e

# --- Main Logic (Sequential Agents) ---

def main():
    """
    Concept 1: Sequential Agents - Main conversational loop with explicit state tracking.
    """
    global current_step
    
    # Store conversation history only for the next immediate turn (more efficient than storing everything)
    chat_history_buffer = []

    print("=" * 60)
    print("🤖 Welcome to the AI Personalized Diet Planner Console!")
    print("✨ Features: Sequential Agents, Web Grounding (Tools), and Context Compaction (Memory).")
    print("=" * 60)
    
    # --- Step 1: Get the initial question ---
    print("\nAI: Hello! I'm your AI Diet Planner, ready to create a personalized plan just for you. I'm starting the 6-step consultation now.")
    
    # The first question is always the first one in the sequence
    first_question = QUESTION_SEQUENCE[0]
    initial_prompt = f"Start the consultation by asking the first question: '{first_question}'"
    
    # Use the conversation system instruction
    try:
        chat_history_buffer.append({"role": "user", "parts": [{"text": initial_prompt}]})
        response_text, _ = generate_content_with_retry(chat_history_buffer, CONVERSATION_SYSTEM_INSTRUCTION)
        
        # Clear buffer and store model response to start the actual history
        chat_history_buffer.clear()
        chat_history_buffer.append({"role": "model", "parts": [{"text": response_text}]})

        print("\n" + "="*20 + " Consultation Started " + "="*20)
        print(f"\nAI: {response_text}\n")
        
    except Exception as e:
        print("\nCRITICAL ERROR: Could not start the consultation. Check API Key.")
        print(f"Details: {e}")
        return

    # --- Step 2: Main Sequential Conversation Loop ---
    while current_step < len(QUESTION_SEQUENCE):
        try:
            # 1. Get user input
            user_input = input("You: ")
            if not user_input.strip():
                continue
            
            # 2. Store the user's answer into the compacted memory structure
            user_answers[QUESTION_SEQUENCE[current_step]] = user_input
            
            # 3. Increment the step counter (Crucial for sequential flow)
            current_step += 1
            
            if current_step >= len(QUESTION_SEQUENCE):
                # All questions answered, time to generate the final plan!
                print("\nAI: Thank you for providing all the necessary details!")
                break

            # 4. Prepare for the next question turn
            next_question = QUESTION_SEQUENCE[current_step]
            
            # The prompt now tells the model what the last answer was and what to ask next
            chat_history_buffer.append({"role": "user", "parts": [{"text": f"User answered the previous question: '{user_input}'. Now, please acknowledge the answer and ask the next question in the sequence: '{next_question}'"}]})
            
            print("AI: Thinking...")
            
            # 5. Get response from AI (using the conversational system instruction)
            model_response_text, _ = generate_content_with_retry(chat_history_buffer, CONVERSATION_SYSTEM_INSTRUCTION)

            # 6. Display and update history buffer for the next turn
            print(f"\nAI: {model_response_text}\n")
            
            # Keep only the last turn for the buffer to save context length/cost
            chat_history_buffer = [{"role": "model", "parts": [{"text": model_response_text}]}]


        except KeyboardInterrupt:
            print("\nConversation interrupted by user. Exiting.")
            return
        except Exception as e:
            print(f"\nAn error occurred during the chat turn: {e}. Please restart or check your configuration.")
            return

    # --- Step 3: Final Plan Generation (Using Context Compaction & Grounding) ---
    
    print("\n" + "="*20 + " Generating Personalized Plan " + "="*20)
    print("AI: Analyzing your data and consulting the latest nutritional guidelines (using Google Search grounding)... This may take a moment.")

    # 1. Compact the context (Memory)
    final_context = compact_context()
    
    # 2. Prepare the final prompt (with the compacted context)
    final_prompt = PLAN_GENERATION_PROMPT.format(compacted_context=final_context)
    
    # 3. Call the API (using Grounding Tool)
    try:
        # Pass the final prompt as the only history item
        final_history = [{"role": "user", "parts": [{"text": final_prompt}]}]
        final_plan_text, sources = generate_content_with_retry(
            final_history, 
            system_prompt="You are a professional diet planner creating a final report.", 
            use_grounding=True
        )

        print("\n" + "="*15 + " Consultation Complete - Final Plan Generated " + "="*15)
        print("\n" + final_plan_text)
        
        if sources:
            print("\n--- Sources Used (Web Grounding) ---")
            for source in sources:
                print(source)
        
        print("\n" + "="*15 + " End of Plan " + "="*15)


    except Exception as e:
        print("\nCRITICAL ERROR: Failed to generate the final plan.")
        print(f"Details: {e}")

if __name__ == "__main__":
    main()

✅ Gemini API key setup complete.
🤖 Welcome to the AI Personalized Diet Planner Console!
✨ Features: Sequential Agents, Web Grounding (Tools), and Context Compaction (Memory).

AI: Hello! I'm your AI Diet Planner, ready to create a personalized plan just for you. I'm starting the 6-step consultation now.

==================== Consultation Started ====================

AI: Hello! I'm happy to help you with your nutrition and diet planning.

Let's start with the first piece of information.

**Question 1 of 6: Initial Metrics (Age, Height, Weight, Gender)**

Please provide your age, height, current weight, and gender.



You:  55,5.1,75,women


AI: Thinking...

AI: That's great, thank you for providing your initial metrics (55 years old, 5'1", 75 kg, female).

**Question 2 of 6: Main Goal**

What is your primary goal? Are you looking for weight loss, weight gain, weight maintenance, or focusing primarily on general health/wellness?



You:  weight loss


AI: Thinking...

AI: Got it, weight loss is a fantastic goal!

**Question 3 of 6: Activity Level**

To accurately calculate your needs, how would you categorize your current physical activity level?

*   **Sedentary:** Little to no exercise.
*   **Lightly Active:** Light exercise or sports 1–3 days/week.
*   **Moderately Active:** Moderate exercise or sports 3–5 days/week.
*   **Very Active:** Hard exercise every day or twice per day.



...

AI: Thinking...

AI: Being vegetarian is a key piece of information for meal planning!

**Question 5 of 6: Preferences & Dislikes**

To make sure your plan is enjoyable, please list 2-3 of your favorite foods/ingredients and 2-3 foods/ingredients you strongly dislike or refuse to eat.



You:  like all, not like eggs and non veg


AI: Thinking...

AI: Got it! That's a great starting point—knowing you dislike eggs and are open to most other vegetarian options gives me a lot of flexibility.

We're on the final step!

**Question 6 of 6: Cooking Style**

Finally, tell me a little about your cooking habits. Are you looking for quick meals (under 30 minutes), or do you prefer to spend more time cooking? Do you like to meal prep for the week, or do you prefer fresh meals daily?



You:  1hr



AI: Thank you for providing all the necessary details!

==================== Generating Personalized Plan ====================
AI: Analyzing your data and consulting the latest nutritional guidelines (using Google Search grounding)... This may take a moment.

=============== Consultation Complete - Final Plan Generated ===============

This report outlines a personalized 7-day diet and wellness plan designed to support your weight loss goal, factoring in your age, activity level, vegetarian preference, and time constraints for cooking. This plan is data-driven, prioritizing a sustainable calorie deficit and high-protein intake to preserve muscle mass and enhance satiety.

---

## **Section 1: Summary & Estimated Metrics**

Your personalized plan is built upon a calculated calorie target that promotes safe and sustainable weight loss of approximately 0.5 kg (1 lb) per week.

### **Estimated Energy Expenditure & Calorie Target**

| Metric | Calculation / Rationale | Estimated Value |
| 